In [2]:
import requests
from bs4 import BeautifulSoup
url='https://books.toscrape.com/index.html'
response=requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')
soup

<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="s

In [3]:
ratings_dic={'One':1,'Two':2,'Three':3,'Four':4,'Five':5}

In [4]:
import sqlite3

In [10]:
import csv
from functools import reduce
domain='https://books.toscrape.com/'
for cat in soup.find('ul',attrs={'class':''}).find_all('li'):
    cat_name=cat.find('a').get_text().strip()
    all_url=domain+cat.find('a').get('href')
    cat_soup_final=BeautifulSoup(requests.get(all_url).text,'html.parser')
        
    for article in cat_soup_final.find_all('article',attrs={'class':'product_pod'}):
        bookname=article.find('h3').find('a').get('title')
        price=float(article.find_all('p')[1].get_text()[2:])
        rate=ratings_dic[article.find('p').get('class')[1]]
        category=cat_name
        connection=sqlite3.connect('lastbookcoll.db')
        cursor=connection.execute('insert into lastbooks values(?,?,?,?)',[bookname,price,rate,category])
        connection.commit()
        connection.close()
        while cat_soup_final.find('li',attrs={'class':'next'}):
                new_page_url=reduce(lambda a,b:a+'/'+b,all_url.split('/')[:-1])+'/'+cat_soup_final.find('li',attrs={'class':'next'}).find('a').get('href')
                cat_soup_final=BeautifulSoup(requests.get(new_page_url).text,'html.parser')
                    
                for article in cat_soup_final.find_all('article',attrs={'class':'product_pod'}):
                    bookname=article.find('h3').find('a').get('title')
                    price=float(article.find_all('p')[1].get_text()[2:])
                    rate=ratings_dic[article.find('p').get('class')[1]]
                    category=cat_name
                    connection=sqlite3.connect('lastbookcoll.db')
                    cursor=connection.execute('insert into lastbooks values(?,?,?,?)',[bookname,price,rate,category])
                    connection.commit()
                    connection.close()